In [1]:
import pandas as pd
from pathlib import Path
%load_ext pyprql.magic

In [2]:
disoccupied_df = pd.read_csv(Path('raw_files') / 'csv' / 'disoccupied.csv')
household_data_df = pd.read_csv(Path('raw_files') / 'csv' / 'household_data.csv')
occupied_df = pd.read_csv(Path('raw_files') / 'csv' / 'occupied.csv')
other_workers_df = pd.read_csv(Path('raw_files') / 'csv' / 'other_workers.csv')
generalities_df = pd.read_csv(Path('raw_files') / 'csv' / 'generalities.csv')
migration_df = pd.read_csv(Path('raw_files') / 'csv' / 'migration.csv')
other_income_and_taxes_df = pd.read_csv(Path('raw_files') / 'csv' / 'other_income_and_taxes.csv')
workforce_df = pd.read_csv(Path('raw_files') / 'csv' / 'workforce.csv')

/var/folders/l4/0_7m43593g9bhm_lxx4lpk_80000gq/T/ipykernel_74268/2622697029.py:3: DtypeWarning: Columns (28,89,135,161) have mixed types. Specify dtype option on import or set low_memory=False.
  occupied_df = pd.read_csv(Path('raw_files') / 'csv' / 'occupied.csv')
/var/folders/l4/0_7m43593g9bhm_lxx4lpk_80000gq/T/ipykernel_74268/2622697029.py:4: DtypeWarning: Columns (71) have mixed types. Specify dtype option on import or set low_memory=False.
  other_workers_df = pd.read_csv(Path('raw_files') / 'csv' / 'other_workers.csv')
/var/folders/l4/0_7m43593g9bhm_lxx4lpk_80000gq/T/ipykernel_74268/2622697029.py:8: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  workforce_df = pd.read_csv(Path('raw_files') / 'csv' / 'workforce.csv')


In [3]:
%prql duckdb:///:memory:
%prql --persist disoccupied_df
%prql --persist household_data_df
%prql --persist occupied_df
%prql --persist other_workers_df
%prql --persist generalities_df
%prql --persist migration_df
%prql --persist other_income_and_taxes_df
%prql --persist workforce_df

In [ ]:
%%prql
from occupied_df